<a href="https://colab.research.google.com/github/comojin1994/Deep_Learning_Study/blob/master/Kaggle/Global_Wheat_Detection/Global_Wheat_Detection_Pseudo_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd ./drive/My\ Drive/Kaggle/Global_Wheat_Detection/input/global-wheat-detection/

/content/drive/My Drive/Kaggle/Global_Wheat_Detection/input/global-wheat-detection


In [3]:
%cd ..

/content/drive/My Drive/Kaggle/Global_Wheat_Detection/input


In [4]:
%cd ..

/content/drive/My Drive/Kaggle/Global_Wheat_Detection


In [5]:
%cd output/

/content/drive/My Drive/Kaggle/Global_Wheat_Detection/output


In [6]:
!ls

Data_Augmentation_Tutorial.ipynb
Global_Wheat_Detection_FasterRCNN_resnet50fpn.ipynb
Global_Wheat_Detection_FasterRCNN_resnet50_Inference.ipynb
Global_Wheat_Detection_FasterRCNN_ResNext101.ipynb
Global_Wheat_Detection_Pseudo_Labeling.ipynb


# Load Module

In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from tqdm import tqdm
import seaborn as sns

import cv2

import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision import transforms

import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import random

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
os.listdir('../input/global-wheat-detection/test/')

test_dir = '../input/global-wheat-detection/test/'
train_dir = '../input/global-wheat-detection/train/'

test_df = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# Callbacks

In [0]:
def checkpoints(best_loss, current_loss, epoch, initalization=True):
    if initalization:
        if epoch < 5:
            return max(best_loss, current_loss)

    if best_loss > current_loss:
        best_loss = current_loss
        torch.save(model.state_dict(), f'../input/checkpoints/best{epoch}_{best_loss}.pth')
    return best_loss

def earlyStopping(best_loss, current_loss, cnt, threshold=10):
    flag = False
    if best_loss < current_loss:
        cnt += 1
    else:
        cnt = 0

    if cnt >= threshold:
        flag = True
    return cnt, flag

# DataLoader

In [0]:
def get_transform():
  list_transforms = []
  list_transforms.append(transforms.ToTensor())
  return transforms.Compose(list_transforms)

In [0]:
class PsuedoWheat(torch.utils.data.Dataset):
  def __init__(self, input_df, input_dir, transforms=None):
    
    self.df = input_df
    self.list_images = list(self.df['image_id'].unique())
    self.image_dir = input_dir
    self.transforms = transforms

  def __getitem__(self, idx):
    img_id = self.list_images[idx]

    img = cv2.imread(self.image_dir + '/' + img_id + '.jpg')
    im_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_scaled = img / 255.
    img_final = img_scaled

    if self.transforms is not None:
      img_final = self.transforms(img_final)

    return img_final, img_id

  def __len__(self):
    return len(self.df['image_id'].unique())

In [0]:
class GlobalWheatDetectionDataset(torch.utils.data.Dataset):
  def __init__(self, input_df, input_dir, transforms=None):

    self.df = input_df
    self.list_images = list(self.df['image_id'].unique())
    self.image_dir = input_dir
    self.transforms = transforms

  def __getitem__(self, idx):
    cols_to_be_selected = ['x_min', 'y_min', 'x_max', 'y_max']
    img_id = self.list_images[idx]
    bboxes_array = np.array(self.df.loc[self.df['image_id'] == img_id, cols_to_be_selected])
    boxes = torch.tensor(bboxes_array, dtype=torch.int64)

    num_boxes = self.df.loc[self.df['image_id'] == img_id].shape[0]
    labels = torch.ones(num_boxes, dtype=torch.int64)

    area = torch.tensor(np.array(self.df['area']), dtype=torch.int64)

    iscrowd = torch.zeros(num_boxes, dtype=torch.uint8)

    target = {}
    target['boxes'] = boxes
    target['labels'] = labels
    target['area'] = area
    target['iscrowd'] = iscrowd

    if 'nvnn' in img_id:
      img_id = img_id[4:]
      img = cv2.imread(f'{test_dir}/{img_id}.jpg')
    else:
      img = cv2.imread(f'{self.image_dir}/{img_id}.jpg')
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_scaled = img_RGB / 255.
    img_final = img_scaled

    if self.transforms is not None:
      img_final = self.transforms(img_final)

    return img_final, target, img_id
  
  def __len__(self):
    return len(self.df['image_id'].unique())

In [0]:
test_dataset = PsuedoWheat(test_df, test_dir, get_transform())

In [0]:
def collate_fn(batch):
  return tuple(zip(*batch))

In [0]:
test_dataloader = DataLoader(
    test_dataset,
    batch_size = 2,
    shuffle = False,
    num_workers = 4,
    collate_fn = collate_fn
)

# Model

In [0]:
def fasterrcnn_fpn(pretrained=False, progress=True,
                   num_classes=2, pretrained_backbone=True,
                   trainable_backbone_layers=3, **kwargs):
  assert trainable_backbone_layers <= 5 and trainable_backbone_layers >= 0
  if not (pretrained or pretrained_backbone):
    trainable_backbone_layers = 5
  if pretrained:
    pretrained_backbone = False
  backbone = resnet_fpn_backbone('resnet152', pretrained_backbone)
  model = FasterRCNN(backbone, num_classes, **kwargs)
  return model

In [0]:
def initialize_model(pretrained):
  model = fasterrcnn_fpn(pretrained=pretrained)
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
  return model

In [0]:
model = initialize_model(pretrained=True)

In [0]:
saved_model_path = '../input/fasterrcnn/FasterRCNN_ResNet152_FPN_Best_49epoch.pth'

In [21]:
model.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [22]:
torch.cuda.empty_cache()
model.to(device)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

# Make Pseudo labels

In [0]:
def format_prediction_string(boxes, scores):
  pred_strings = []
  for j in zip(scores, boxes):
    pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [0]:
detection_threshold = 0.5
results = []
testdf_psuedo = []

In [25]:
for images, image_ids in test_dataloader:
  
  images = list(image.to(device, dtype=torch.float) for image in images)
  outputs = model(images)
  
  for i, image in enumerate(images):
    boxes = outputs[i]['boxes'].data.cpu().numpy()
    scores = outputs[i]['scores'].data.cpu().numpy()

    boxes = boxes[scores >= detection_threshold].astype(np.int32)
    scores = scores[scores >= detection_threshold]

    image_id = image_ids[i]

    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    
    for box in boxes:
      result = {
          'image_id': 'nvnn'+image_id,
          'x_min': box[0],
          'y_min': box[1],
          'x_max': box[0] + box[2],
          'y_max': box[1] + box[3],
          'width': box[2],
          'height': box[3],
          'area': box[2] * box[3],
          'source': 'nvnn',
      }
      testdf_psuedo.append(result)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [26]:
test_df_psuedo = pd.DataFrame(testdf_psuedo,
                              columns=['image_id', 'x_min', 'y_min',
                                       'x_max', 'y_max', 'width',
                                       'height', 'area', 'source'])
test_df_psuedo.head()

,image_id,x_min,y_min,x_max,y_max,width,height,area,source
0,nvnnaac893a91,61,0,176,158,115,158,18170,nvnn
1,nvnnaac893a91,612,917,696,1020,84,103,8652,nvnn
2,nvnnaac893a91,738,769,817,887,79,118,9322,nvnn
3,nvnnaac893a91,692,381,819,557,127,176,22352,nvnn
4,nvnnaac893a91,329,660,449,810,120,150,18000,nvnn


In [0]:
train_df = pd.read_csv('../input/global-wheat-detection/train.csv')

In [0]:
train_df = train_df[train_df['height'] > 10]
train_df = train_df[train_df['width'] > 10]

In [0]:
frames = [train_df, test_df_psuedo]

train_df = pd.concat(frames)

In [0]:
train_df = train_df.reset_index(drop=True)

# DataLoader

In [0]:
val_percentage = 0.2
np.random.seed(1524)
list_val_imageid = np.random.choice(np.array(train_df['image_id'].unique()),
                 int(len(train_df['image_id'].unique()) * val_percentage))
list_train_imageid = np.setdiff1d(np.array(train_df['image_id'].unique()), list_val_imageid)

In [47]:
print('Number of validation images: ', len(list_val_imageid))
print('Number of training images: ', len(list_train_imageid))
print('Number of total images: ', len(train_df['image_id'].unique()))
print('Check: ', len(train_df['image_id'].unique()) == len(list_val_imageid) + len(list_train_imageid))

Number of validation images:  1351
Number of training images:  5550
Number of total images:  6755
Check:  False


In [0]:
df_val = train_df.loc[train_df['image_id'].isin(list_val_imageid), :]
df_train = train_df.loc[train_df['image_id'].isin(list_train_imageid), :]

In [0]:
train_dataset = GlobalWheatDetectionDataset(df_train, train_dir, get_transform())
val_dataset = GlobalWheatDetectionDataset(df_val, train_dir, get_transform())

In [0]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size = 8,
    shuffle = True,
    num_workers = 4,
    collate_fn = collate_fn
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = 8,
    shuffle = False,
    num_workers = 4,
    collate_fn = collate_fn
)

# Training

In [0]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.00005, max_lr=0.005)
num_epochs = 100

In [0]:
itr = 1
best_loss = np.float('inf')
flag = False
cnt = 0
hist_train_loss = []
hist_val_loss = []
initial_epoch = 1

In [0]:
for epoch in range(initial_epoch, num_epochs + 1):
  model.train()
  loss_sum = 0
  num_iterations = 0
  for images, targets, image_ids in tqdm(train_dataloader):

    